In [6]:



from BenchmarkProblems.Checkerboard import CheckerBoard
from BenchmarkProblems.GraphColouring import GraphColouring
from BenchmarkProblems.Trapk import Trapk
from Core.get_pRef import get_pRef_from_metaheuristic
# Check that reference populations and metrics work as intended

from Core.FullSolution import FullSolution
from Core.PS import PS, contains, STAR
from BenchmarkProblems.RoyalRoad import RoyalRoad

# first, we make a benchmark problem
problem = RoyalRoad(5)

# then we make a pRef
pRef = get_pRef_from_metaheuristic(problem=problem,
                                   sample_size=10000,
                                   which_algorithm="GA uniform",
                                   unique=True,
                                   verbose=True)

# we can remove duplicated_entries (recommended)


print(f"Generated the PRef: {pRef}")

Running the algorithm to generate the PRef using GA......Finished (took 0.542422 seconds)
Running the algorithm to generate the PRef using uniform......Finished (took 0.288718 seconds)
Generated the PRef: PRef with 5759 samples, mean = 2.18


In [7]:


from Core.PRef import PRef
from typing import Optional
from Core.EvaluatedFS import EvaluatedFS
import numpy as np


def get_quality(wins: int, losses: int) -> Optional[float]:
    if wins+losses == 0:
        return -1
    else:
        return wins / (wins+losses)
    


def add_pss(ps_a: PS, ps_b: PS) -> PS:
    return PS(from_a if from_a != STAR else from_b for from_a, from_b in zip(ps_a.values, ps_b.values))


def minus_pss(ps_a: PS, ps_b: PS) -> PS:
    return PS(from_a if from_b == STAR else STAR for from_a, from_b in zip(ps_a.values, ps_b.values))
    
    
def get_wins_and_losses_for_solution_and_ps(solution: EvaluatedFS, ps: PS, non_matching_pRef: PRef) -> (int, int):
    # find the solutions that are similar to s, but without the pattern ps
    pattern_solution_minus_ps = minus_pss(PS.from_FS(solution), ps)
    
    fitnesses_without_ps = non_matching_pRef.fitnesses_of_observations(pattern_solution_minus_ps)
    if len(fitnesses_without_ps) == 0:
        return (0, 0)
    

    wins = np.sum(fitnesses_without_ps < solution.fitness)
    losses = np.sum(fitnesses_without_ps > solution.fitness)
    return wins, losses


def get_total_wins_and_losses_for_ps(ps: PS):
    wins, losses = (0,0)
    indexes_of_ps = pRef.get_indexes_matching_ps(ps)
    matching_pRef, non_matching_pRef = pRef.split_by_indexes(indexes_of_ps)
    for solution in matching_pRef.get_evaluated_FSs():
        new_wins, new_losses = get_wins_and_losses_for_solution_and_ps(solution, ps, non_matching_pRef)
        wins += new_wins
        losses += new_losses
        
    return wins, losses


def get_quality_of_ps(ps: PS) -> float:
    wins, losses = get_total_wins_and_losses_for_ps(ps)
    
    if wins+losses == 0:
        return -1
    
    #before = wins / (wins+losses)
    after = wins / (wins+losses+1)
    return after

    

In [8]:
random_pss = [PS.random_with_fixed_size(problem.search_space, size=5) for _ in range(100)]

random_pss.extend(problem.get_targets())

random_pss.sort(key=get_quality_of_ps, reverse=True)

for ps in random_pss:
    qual = get_quality_of_ps(ps)
    wins, losses = get_total_wins_and_losses_for_ps(ps)
    print(f"The ps is {ps}, it has quality {qual}, {wins =}, {losses = }")
    print(problem.repr_ps(ps))
    
    

The ps is [1 1 1 1 * * * * * * * * * * * * * * * *], it has quality 0.996078431372549, wins =254, losses = 0
[1 1 1 1  * * * *  * * * *  * * * *  * * * *]
The ps is [* * * * * * * * * 1 1 1 * * * * * 1 1 *], it has quality 0.996, wins =249, losses = 0
[* * * *  * * * *  * 1 1 1  * * * *  * 1 1 *]
The ps is [* * * * * * * * * * * * * * * * 1 1 1 1], it has quality 0.9959183673469387, wins =244, losses = 0
[* * * *  * * * *  * * * *  * * * *  1 1 1 1]
The ps is [* * * * * * * * * * * * * 1 1 1 1 * * 1], it has quality 0.9957264957264957, wins =233, losses = 0
[* * * *  * * * *  * * * *  * 1 1 1  1 * * 1]
The ps is [* * * * * * * * 1 1 1 1 * * * * * * * *], it has quality 0.9955947136563876, wins =226, losses = 0
[* * * *  * * * *  1 1 1 1  * * * *  * * * *]
The ps is [* * * * * * * * * * * * 1 1 1 1 * * * *], it has quality 0.9954954954954955, wins =221, losses = 0
[* * * *  * * * *  * * * *  1 1 1 1  * * * *]
The ps is [* * * * 1 1 1 1 * * * * * * * * * * * *], it has quality 0.99541284

In [5]:



from SimplifiedSystem.ps_search_utils import run_pymoo_algorithm_with_checks
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.algorithms.soo.nonconvex.ga import GA
from SimplifiedSystem.Operators.Mutation import GlobalPSUniformMutation
from SimplifiedSystem.Operators.Crossover import GlobalPSUniformCrossover
from SimplifiedSystem.Operators.Sampling import GlobalPSGeometricSampling
from SimplifiedSystem.GlobalPSSearchTask import find_ps_in_problem, GlobalPSSearchTask
from SimplifiedSystem.search_methods import search_global_ps

def inverse_quality(ps: PS):
    return -get_quality_of_ps(ps)

def simplicity(ps: PS):
    return -np.sum(ps.values == STAR)
objectives = [simplicity, inverse_quality]

# construct the optimisation problem instance
pymoo_problem = GlobalPSSearchTask(
    original_problem_search_space=problem.search_space,
    objectives=objectives)

# if there are no operators given, we have these defaults
sampling_operator = GlobalPSGeometricSampling()
crossover_operator = GlobalPSUniformCrossover(prob=0.3)
mutation_operator = GlobalPSUniformMutation(prob=1 / pymoo_problem.n_var,search_space=problem.search_space) 

# the next line of code is a bit odd, but it works! It uses a GA if there is one objective
algorithm = (GA if len(objectives) < 2 else NSGA2)(pop_size=100,
                                                   sampling=sampling_operator,
                                                   crossover=crossover_operator,
                                                   mutation=mutation_operator,
                                                   eliminate_duplicates=True)

pss = run_pymoo_algorithm_with_checks(pymoo_problem=pymoo_problem,
                                      algorithm=algorithm,
                                      reattempts_when_fail=1,
                                      ps_budget=1000,
                                      verbose=True)




C:\Users\gac8\AppData\Local\anaconda3\Lib\site-packages\numpy\lib\function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
C:\Users\gac8\AppData\Local\anaconda3\Lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


n_gen  |  n_eval  | n_nds  |     cv_min    |     cv_avg    |      eps      |   indicator  
     1 |       82 |      6 |  0.000000E+00 |  0.000000E+00 |             - |             -
     2 |      182 |      8 |  0.000000E+00 |  0.000000E+00 |  0.0604739392 |             f
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\gac8\AppData\Local\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\gac8\AppData\Local\Temp\ipykernel_5420\725873748.py", line 34, in <module>
    pss = run_pymoo_algorithm_with_checks(pymoo_problem=pymoo_problem,
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\gac8\PycharmProjects\PSSearch\SimplifiedSystem\ps_search_utils.py", line 146, in run_pymoo_algorithm_with_checks
    pss = run_and_get_results()
          ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\gac8\PycharmProjects\PSSearch\SimplifiedSystem\ps_search_utils.py", line 122, in run_and_get_results
    res = minimize(pymoo_problem,
          ^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\gac8\AppData\Local\anaconda3\Lib\site-packages\pymoo\optimize.py", line 67, in minimize
    res = algorithm.run()
          ^^^^^^^^^^^^^^^
  File "C:\Users\g

In [ ]:
import utils

pss.sort(key=lambda ps: ps.metric_scores[1])
simpl = [-ps.metric_scores[0] for ps in pss]
qual = [-ps.metric_scores[1] for ps in pss]


def get_diffs(input_list):
    return [p-h for h, p in zip(input_list[1:], input_list)]

first_derivative = get_diffs(qual)
second_derivative = get_diffs(first_derivative)
first_derivative = [None] +first_derivative
second_derivative = [None, None] + second_derivative
for ps, fd, sd in zip(pss, first_derivative, second_derivative):
    print(ps, fd, sd)
    

from utils import simple_scatterplot


simple_scatterplot(x_label="simpl", y_label = "quality", xs = simpl, ys = qual)

